In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

# with the new api
from importnb import imports
with imports("ipynb"):
    from utils import df_from_csv, to_be_checked, is_corr_ok, plot_hist, plot_boxes, plot_lines, df_to_csv, drop_pm25_outliers

PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## PM2.5 Clean Data

In [2]:
# Load Raw Data
df_pm25_raw = df_from_csv("pm25_raw.csv")

# Check Data Types
print(df_pm25_raw.dtypes)

df_pm25_raw.head()

TANGARA_260A    float64
TANGARA_2BDE    float64
TANGARA_39D6    float64
TANGARA_3B7E    float64
TANGARA_3BEA    float64
TANGARA_421A    float64
TANGARA_5D62    float64
TANGARA_5636    float64
TANGARA_D282    float64
TANGARA_4B1A    float64
TANGARA_14D6    float64
TANGARA_2B42    float64
TANGARA_2E9A    float64
TANGARA_2FF6    float64
TANGARA_307A    float64
TANGARA_48C6    float64
TANGARA_F1AE    float64
TANGARA_06BE    float64
dtype: object


,TANGARA_260A,TANGARA_2BDE,TANGARA_39D6,TANGARA_3B7E,TANGARA_3BEA,TANGARA_421A,TANGARA_5D62,TANGARA_5636,TANGARA_D282,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,,,,,,,,,
2023-11-17 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,8.0,NaN
2023-11-17 00:00:30-05:00,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,7.0,10.0,5.0,9.0,10.0,4.0,7.0,9.0
2023-11-17 00:01:00-05:00,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,9.0,10.0,7.0,8.0,10.0,8.0,7.0,9.0
2023-11-17 00:01:30-05:00,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,10.0,11.0,11.0,7.0,10.0,6.0,8.0,8.0
2023-11-17 00:02:00-05:00,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,9.0,11.0,10.0,8.0,10.0,5.0,7.0,8.0


## Descriptive Statistics

In [3]:
# Describe Data
df_pm25_raw.describe()

,TANGARA_260A,TANGARA_2BDE,TANGARA_39D6,TANGARA_3B7E,TANGARA_3BEA,TANGARA_421A,TANGARA_5D62,TANGARA_5636,TANGARA_D282,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
count,5727.00000,1359.000000,1.0,1292.000000,1376.000000,1425.000000,2175.000000,1304.000000,1435.000000,5726.000000,5665.000000,4978.000000,4810.000000,5718.000000,5711.000000,5733.000000,5739.000000,5715.000000
mean,8.64938,5.729213,19.0,4.528638,4.756541,4.750877,5.322299,6.046779,5.514286,11.699616,9.561342,8.166533,7.926403,7.883176,9.531956,7.238095,8.528664,9.628696
std,5.12304,3.547318,NaN,2.849156,3.045765,3.036131,4.110204,3.497411,3.319195,16.372879,5.945327,5.151530,5.814738,4.034657,13.733579,7.289115,7.589274,4.943717
min,1.00000,1.000000,19.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
25%,5.00000,3.000000,19.0,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,5.000000,5.000000,3.000000,5.000000,5.000000,3.000000,4.000000,6.000000
50%,8.00000,5.000000,19.0,4.000000,4.000000,4.000000,4.000000,5.000000,5.000000,8.000000,9.000000,8.000000,7.000000,8.000000,8.000000,6.000000,7.000000,9.000000
75%,11.00000,7.000000,19.0,6.000000,6.000000,6.000000,6.000000,8.000000,7.000000,15.000000,13.000000,10.000000,11.000000,10.000000,10.000000,9.000000,11.000000,12.000000
max,33.00000,18.000000,19.0,18.000000,18.000000,17.000000,63.000000,17.000000,18.000000,310.000000,57.000000,95.000000,37.000000,22.000000,339.000000,195.000000,264.000000,61.000000


## Missing Data

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_pm25_raw.columns:
    df_pm25_sensor = df_pm25_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_pm25_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%
Tangara Sensor: TANGARA_2BDE, Data: 24%, Missing: 76%, To be checked
Tangara Sensor: TANGARA_39D6, Data: 0%, Missing: 100%, To be checked
Tangara Sensor: TANGARA_3B7E, Data: 22%, Missing: 78%, To be checked
Tangara Sensor: TANGARA_3BEA, Data: 24%, Missing: 76%, To be checked
Tangara Sensor: TANGARA_421A, Data: 25%, Missing: 75%, To be checked
Tangara Sensor: TANGARA_5D62, Data: 38%, Missing: 62%, To be checked
Tangara Sensor: TANGARA_5636, Data: 23%, Missing: 77%, To be checked
Tangara Sensor: TANGARA_D282, Data: 25%, Missing: 75%, To be checked
Tangara Sensor: TANGARA_2B42, Data: 86%, Missing: 14%, To be checked
Tangara Sensor: TANGARA_2E9A, Data: 84%, Missing: 16%, To be checked


## Data Correlation

In [5]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_pm25_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_pm25_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
if not math.isnan(df_reference_sensor.std()):
    is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
    if not is_ok:
        print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_pm25_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_pm25_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")

Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


## Histograms

In [6]:
# Plot Histograms
plot_hist(df_pm25_raw) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Boxplots

In [7]:
# Plot Boxplots
plot_boxes(df_pm25_raw) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Lineplots

In [8]:
# Plot Lineplots
plot_lines(df_pm25_raw) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Drop PM2.5 Outliers

In [9]:
# Drop PM2.5 Outliers
df_pm25_clean, resume = drop_pm25_outliers(df_pm25_raw)

In [10]:
# Describe Data
df_pm25_raw.describe()

,TANGARA_260A,TANGARA_2BDE,TANGARA_39D6,TANGARA_3B7E,TANGARA_3BEA,TANGARA_421A,TANGARA_5D62,TANGARA_5636,TANGARA_D282,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
count,5592.000000,1288.000000,1.0,1267.000000,1285.000000,1314.000000,2045.000000,1294.000000,1400.000000,5544.000000,5557.000000,4683.000000,4742.000000,5585.000000,5435.000000,5527.000000,5582.000000,5562.000000
mean,8.286302,5.202640,19.0,4.321231,4.178210,4.088280,4.605379,5.967543,5.275000,9.759740,9.175814,7.331839,7.671447,7.623456,7.764121,6.363307,7.907381,9.238403
std,4.591809,2.810253,NaN,2.447919,2.149076,2.051834,2.018086,3.391979,2.983529,6.905015,5.269880,3.873654,5.438970,3.707354,3.385044,4.104509,4.585627,4.351320
min,1.000000,1.000000,19.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
25%,5.000000,3.000000,19.0,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,5.000000,5.000000,3.000000,5.000000,5.000000,3.000000,4.000000,6.000000
50%,8.000000,4.000000,19.0,4.000000,4.000000,3.000000,4.000000,5.000000,4.000000,8.000000,9.000000,8.000000,7.000000,8.000000,8.000000,6.000000,7.000000,9.000000
75%,11.000000,7.000000,19.0,6.000000,5.000000,5.000000,6.000000,8.000000,7.000000,14.000000,12.000000,10.000000,10.000000,10.000000,10.000000,9.000000,11.000000,12.000000
max,20.000000,13.000000,19.0,12.000000,10.000000,10.000000,10.000000,15.000000,13.000000,31.000000,25.000000,17.000000,23.000000,17.000000,17.000000,18.000000,21.000000,21.000000


In [11]:
# Describe Data
df_pm25_clean.describe()

,TANGARA_260A,TANGARA_2BDE,TANGARA_39D6,TANGARA_3B7E,TANGARA_3BEA,TANGARA_421A,TANGARA_5D62,TANGARA_5636,TANGARA_D282,TANGARA_4B1A,TANGARA_14D6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_48C6,TANGARA_F1AE,TANGARA_06BE
count,5592.000000,1288.000000,1.0,1267.000000,1285.000000,1314.000000,2045.000000,1294.000000,1400.000000,5544.000000,5557.000000,4683.000000,4742.000000,5585.000000,5435.000000,5527.000000,5582.000000,5562.000000
mean,8.286302,5.202640,19.0,4.321231,4.178210,4.088280,4.605379,5.967543,5.275000,9.759740,9.175814,7.331839,7.671447,7.623456,7.764121,6.363307,7.907381,9.238403
std,4.591809,2.810253,NaN,2.447919,2.149076,2.051834,2.018086,3.391979,2.983529,6.905015,5.269880,3.873654,5.438970,3.707354,3.385044,4.104509,4.585627,4.351320
min,1.000000,1.000000,19.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
25%,5.000000,3.000000,19.0,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,5.000000,5.000000,3.000000,5.000000,5.000000,3.000000,4.000000,6.000000
50%,8.000000,4.000000,19.0,4.000000,4.000000,3.000000,4.000000,5.000000,4.000000,8.000000,9.000000,8.000000,7.000000,8.000000,8.000000,6.000000,7.000000,9.000000
75%,11.000000,7.000000,19.0,6.000000,5.000000,5.000000,6.000000,8.000000,7.000000,14.000000,12.000000,10.000000,10.000000,10.000000,10.000000,9.000000,11.000000,12.000000
max,20.000000,13.000000,19.0,12.000000,10.000000,10.000000,10.000000,15.000000,13.000000,31.000000,25.000000,17.000000,23.000000,17.000000,17.000000,18.000000,21.000000,21.000000


## Histograms

In [12]:
# Plot Histograms
plot_hist(df_pm25_clean) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Boxplots

In [13]:
# Plot Boxplots
plot_boxes(df_pm25_clean) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


## Lineplots

In [14]:
# Plot Lineplots
plot_lines(df_pm25_clean) if bool(os.getenv("PLOT_CHARTS", None)) else print("PLOT_CHARTS:", False, "Plot charts were ignored")

PLOT_CHARTS: False Plot charts were ignored


In [15]:
# Save PM2.5 Data Frame Sensors into CSV file
df_to_csv(df_pm25_clean, "pm25_clean.csv", datafolder='1_clean')